In [2]:
import pandas as pd
from power_grid_model.utils import json_deserialize_from_file
import numpy as np

from power_grid_model import LoadGenType

from power_grid_model import (
    PowerGridModel,
    CalculationMethod,
    CalculationType,
    MeasuredTerminalType,
    BranchSide,
)

from power_grid_model.validation import assert_valid_input_data
from power_grid_model.enum import _ExperimentalFeatures

In [3]:
input_data = json_deserialize_from_file("input_network_data1.json")
active_power_profile1 = pd.read_parquet('active_power_profile1.parquet')
reactive_power_profile1 = pd.read_parquet('reactive_power_profile1.parquet')

In [4]:
print(pd.DataFrame(input_data['node']))

   id  u_rated
0   0  10500.0
1   1    400.0
2   2    400.0
3   3    400.0
4   4    400.0
5   5    400.0
6   6    400.0
7   7    400.0
8   8    400.0
9   9    400.0


In [5]:
print(pd.DataFrame(input_data['line']))

   id  from_node  to_node  from_status  to_status        r1        x1  \
0  16          1        2            1          1  0.000310  0.000141   
1  17          2        3            1          1  0.000052  0.000007   
2  18          2        4            1          1  0.000330  0.000150   
3  19          4        5            1          1  0.000056  0.000008   
4  20          1        6            1          1  0.000316  0.000144   
5  21          6        7            1          1  0.000052  0.000007   
6  22          6        8            1          1  0.000309  0.000140   
7  23          8        9            1          1  0.000062  0.000009   
8  24          4        8            1          0  0.000360  0.000164   

             c1   tan1        r0        x0            c0   tan0           i_n  
0  1.406945e-05  0.003  0.001121  0.000387  8.360108e-06  0.001  4.485912e+06  
1  5.235094e-07  0.003  0.000175  0.000020  3.102278e-07  0.001  9.694619e+04  
2  1.498785e-05  0.003  0.001

In [6]:
print(pd.DataFrame(input_data['transformer']))

   id  from_node  to_node  from_status  to_status       u1     u2        sn  \
0  11          0        1            1          1  10750.0  420.0  630000.0   

      uk      pk  ...  tap_nom  tap_size  uk_min  uk_max  pk_min  pk_max  \
0  0.041  5200.0  ...        3     250.0     NaN     NaN     NaN     NaN   

   r_grounding_from  x_grounding_from  r_grounding_to  x_grounding_to  
0               NaN               NaN             0.0             0.0  

[1 rows x 29 columns]


In [7]:
print(pd.DataFrame(input_data['source']))

   id  node  status  u_ref  u_ref_angle           sk  rx_ratio  z01_ratio
0  10     0       1   1.05          NaN  200000000.0       NaN        NaN


In [8]:
print(pd.DataFrame(input_data['sym_load']))

   id  node  status  type  p_specified  q_specified
0  12     3       1     0          0.0          0.0
1  13     5       1     0          0.0          0.0
2  14     7       1     0          0.0          0.0
3  15     9       1     0          0.0          0.0


In [9]:
from power_system_simulation.power_flow_processing import PowerFlow

power_flow_instance = PowerFlow(input_data)

output_data = power_flow_instance.batch_powerflow(
        active_power_profile=active_power_profile1, reactive_power_profile=reactive_power_profile1
    )

In [10]:
print(pd.DataFrame(output_data["node"]["u"]))

                0           1           2           3           4           5  \
0    11023.097450  429.172586  429.094271  429.087868  429.050534  429.043411   
1    11025.226665  430.364474  430.275539  430.268239  430.231197  430.222774   
2    11020.835073  427.889845  427.796042  427.787993  427.750793  427.743408   
3    11023.342998  429.297456  429.207037  429.199441  429.161970  429.154138   
4    11023.102394  429.169789  429.088442  429.081103  429.049815  429.042594   
..            ...         ...         ...         ...         ...         ...   
955  11022.162034  428.582815  428.472488  428.463784  428.415307  428.405790   
956  11024.927599  430.136428  430.054511  430.047401  430.008609  430.000660   
957  11022.934792  429.049001  428.962020  428.955140  428.915163  428.907747   
958  11022.278198  428.649779  428.554539  428.545965  428.503962  428.495866   
959  11025.586815  430.516104  430.434998  430.427867  430.389585  430.381848   

              6           7

In [11]:
print(input_data['transformer']['from_node'])

[0]


In [33]:
tap_position = 0
number_of_taps_max_min = input_data['transformer']['tap_max']-input_data['transformer']['tap_min']
number_of_taps_min_max = input_data['transformer']['tap_min']-input_data['transformer']['tap_max']
for i in range(len(output_data["node"]["id"])):
    if isinstance(output_data["node"]["id"][i], (list, np.ndarray)):
        for j in range(len(output_data["node"]["id"][i])):
            if output_data["node"]["id"][i][j] == input_data['transformer']['from_node']:
                tap = ((output_data["node"]["u"][i][j]-input_data['transformer']['u1'])/(input_data['transformer']['u1']))*100
                if(input_data['transformer']['tap_max'] < input_data['transformer']['tap_min']):
                    for i in range(number_of_taps_min_max[0]):
                        if((input_data['transformer']['tap_max'])+i-0.5<tap and (input_data['transformer']['tap_max'])+i+0.5>tap):
                            tap = (input_data['transformer']['tap_max'])+i
                            tap_position = tap + tap_position
                elif(input_data['transformer']['tap_max'] > input_data['transformer']['tap_min']):
                    for i in range(number_of_taps_max_min[0]):
                        if((input_data['transformer']['tap_max'])+i-0.5<tap and (input_data['transformer']['tap_max'])+i+0.5>tap):
                            tap = (input_data['transformer']['tap_min'])+i
                            tap_position = tap + tap_position

optimial_tap_position = tap_position/(len(output_data["node"]["id"]))
if(input_data['transformer']['tap_max'] < input_data['transformer']['tap_min']):
    for i in range(number_of_taps_min_max[0]):
        if((input_data['transformer']['tap_max'])+i-0.5<tap and (input_data['transformer']['tap_max'])+i+0.5>tap):
            optimial_tap_position = (input_data['transformer']['tap_max'])+i
elif(input_data['transformer']['tap_max'] > input_data['transformer']['tap_min']):
    for i in range(number_of_taps_max_min[0]):
        if((input_data['transformer']['tap_max'])+i-0.5<tap and (input_data['transformer']['tap_max'])+i+0.5>tap):
            optimial_tap_position = (input_data['transformer']['tap_min'])+i

print(optimial_tap_position[0])

3


In [ ]:
voltage_table = power_flow_instance.aggregate_voltage_table(
    active_power_profile=active_power_profile1, reactive_power_profile=reactive_power_profile1
)

print(voltage_table)

                     Max_Voltage  Max_Voltage_Node  Min_Voltage  \
Timestamp                                                         
2025-01-01 00:00:00     1.072931                 1     1.049819   
2025-01-01 00:15:00     1.075911                 1     1.050022   
2025-01-01 00:30:00     1.069725                 1     1.049603   
2025-01-01 00:45:00     1.073244                 1     1.049842   
2025-01-01 01:00:00     1.072924                 1     1.049819   
...                          ...               ...          ...   
2025-01-10 22:45:00     1.071457                 1     1.049730   
2025-01-10 23:00:00     1.075341                 1     1.049993   
2025-01-10 23:15:00     1.072623                 1     1.049803   
2025-01-10 23:30:00     1.071624                 1     1.049741   
2025-01-10 23:45:00     1.076290                 1     1.050056   

                     Min_Voltage_Node  
Timestamp                              
2025-01-01 00:00:00                 0  
2025-01-

In [35]:
print(pd.DataFrame(output_data['node']["u"]))

                0           1           2           3           4           5  \
0    11023.097450  429.172586  429.094271  429.087868  429.050534  429.043411   
1    11025.226665  430.364474  430.275539  430.268239  430.231197  430.222774   
2    11020.835073  427.889845  427.796042  427.787993  427.750793  427.743408   
3    11023.342998  429.297456  429.207037  429.199441  429.161970  429.154138   
4    11023.102394  429.169789  429.088442  429.081103  429.049815  429.042594   
..            ...         ...         ...         ...         ...         ...   
955  11022.162034  428.582815  428.472488  428.463784  428.415307  428.405790   
956  11024.927599  430.136428  430.054511  430.047401  430.008609  430.000660   
957  11022.934792  429.049001  428.962020  428.955140  428.915163  428.907747   
958  11022.278198  428.649779  428.554539  428.545965  428.503962  428.495866   
959  11025.586815  430.516104  430.434998  430.427867  430.389585  430.381848   

              6           7

In [ ]:
tap_voltage=0
for i in range(len(voltage_table)):
    u_pu_max = voltage_table["Max_Voltage"].iloc[i]
    u_pu_min = voltage_table["Min_Voltage"].iloc[i]
    tap_max = ((u_pu_max-1)/1)*100
    tap_min = ((u_pu_min-1)/1)*100
    tap_voltage = tap_max+tap_min+tap_voltage

tap_position_voltage = tap_voltage/(2*(len(voltage_table)))
print(tap_position_voltage)


6.173418251365674


In [ ]:
loading_table = power_flow_instance.aggregate_loading_table(
    active_power_profile=active_power_profile1, reactive_power_profile=reactive_power_profile1
)

print(loading_table)

         Total_Loss   Max_Loading Max_Loading_Timestamp   Min_Loading  \
Line_ID                                                                 
16        26.709511  6.869324e-05   2025-01-04 06:30:00  1.253601e-05   
17         1.128073  1.653650e-03   2025-01-04 09:45:00  2.697708e-04   
18         9.100636  3.414478e-05   2025-01-07 10:45:00  5.617314e-06   
19         1.220324  1.543576e-03   2025-01-07 10:45:00  2.496785e-04   
20        27.361620  7.086133e-05   2025-01-07 10:45:00  1.172002e-05   
21         1.132925  1.658966e-03   2025-01-07 10:30:00  2.550110e-04   
22         8.584028  3.876018e-05   2025-01-07 10:45:00  5.715455e-06   
23         1.345700  1.500068e-03   2025-01-07 10:45:00  2.202110e-04   
24         2.725758  2.452007e-07   2025-01-10 21:15:00  2.418978e-07   

        Min_Loading_Timestamp  
Line_ID                        
16        2025-01-08 12:30:00  
17        2025-01-08 11:30:00  
18        2025-01-05 17:45:00  
19        2025-01-05 17:45:00  
20  

In [ ]:
transformer_data = output_data["transformer"]
transformer_ids = transformer_data["id"][0, :]

p_from_transformer = pd.DataFrame(transformer_data["p_from"][:, :], columns=transformer_ids)
p_to_transformer = pd.DataFrame(transformer_data["p_to"][:, :], columns=transformer_ids)

p_loss_tranformer = (p_from_transformer + p_to_transformer)

print(p_loss_tranformer)

print(input_data['transformer'])


              11
0    1432.750122
1    1474.464535
2    1478.754012
3    1465.859301
4    1446.208700
..           ...
955  1596.281086
956  1627.554084
957  1512.015776
958  1591.382652
959  1620.175499

[960 rows x 1 columns]
[(11, 0, 1, 1, 1, 10750., 420., 630000., 0.041, 5200., 0.01, 745., 2, 1, 5, 0, 3, 5, 1, 3, 250., nan, nan, nan, nan, nan, nan, 0., 0.)]
